In [1]:
! pip install scikit-surprise

In [7]:
import pandas as pd
import numpy as np
import os
ratings_file = pd.read_csv("ratings_small.csv")

movies_file = pd.read_csv("movies_metadata.csv")
ratings_file.head(5)

C:\Users\lexus\AppData\Local\Temp\ipykernel_2224\3824749267.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_file = pd.read_csv("movies_metadata.csv")


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
from surprise import Dataset, Reader

In [13]:
#using KNN dataset 1st - KNN  is not suitable for large dataset
#Surprise package has a unique format, and the data needs to be converted to surprise format , to be used
#first we read the rating scate , then pass the dataset, along with the rating scale
reader = Reader(rating_scale = (1.0, 5.0))
data = Dataset.load_from_df(ratings_file[["userId", "movieId", "rating"]], reader)
#then we create our trainset from data
train_data = data.build_full_trainset()
train_data

In [15]:
#next using KNN to build our recommender model
#importing the model from surprise
from surprise import KNNWithMeans
#specifying parameters for the model, here we can build either user based or item based CF recommendation system, if we give user_based parameter as false, then the model will compute item based similarity
#we also mention the similarity method to be used
parameters = {"name": "cosine", "user_based" : False}
model = KNNWithMeans(sim_options=parameters)

In [16]:
#fiting our dataset
model.fit(train_data)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [25]:
model.predict(iid = 455, uid = 350).est #predicting the rating based on user id and book id, #est is where the rating is stored.
#we can set up a threshold value, and if the rating is above that threshold value, we can recommend that book to the user

3.3414884050818845

In [29]:
#doing the same with user based = True
parameters = {"name": "cosine", "user_based" : True}
model2 = KNNWithMeans(sim_options=parameters)
model2.fit(train_data)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [30]:
model2.predict(iid = 455, uid = 350).est

3.1286876260204735

In [31]:
model2.predict(iid = 465, uid = 340).est

3.906116114251149

In [32]:
#using KNN baseline
from surprise import KNNBaseline
parameters = {"name": "cosine", "user_based": True}
model3 = KNNBaseline(sim_options=parameters)
model3.fit(train_data)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [34]:
model3.predict(iid = 455, uid = 350).est

3.0474842512490805

In [35]:
model3.predict(iid = 465, uid = 340).est

3.706232729197855

In [42]:
#using SVD Method
#for SVD we give verbose = True, to print all the info
from surprise import SVD
#also we use cross validate, 
from surprise.model_selection import cross_validate
svd = SVD(verbose = True, n_epochs = 10)
#we also set the metrics to root mean square erroe (RMSE), and Mean Absolute Error(MAE)
cross_validate(svd,data,measures=["RMSE", "MAE"], verbose = True, cv = 3 )

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9078  0.9106  0.9016  0.9067  0.0038  
MAE (testset)     0.7018  0.7024  0.6978  0.7007  0.0020  
Fit time          0.50    0.52    0.51    0.51    0.01    
Test time         0.24    0.23    0.50    0.32    0.12    


{'test_rmse': array([0.90777975, 0.91064105, 0.90162051]),
 'test_mae': array([0.70183243, 0.70237017, 0.69779637]),
 'fit_time': (0.497098445892334, 0.5185060501098633, 0.5101230144500732),
 'test_time': (0.2399578094482422, 0.23320770263671875, 0.4987623691558838)}

In [43]:
#next we fir our model
svd.fit(train_data)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [44]:
svd.predict(iid = 12, uid = 16).est

3.319372953618526

In [45]:
svd.predict(iid = 455, uid = 350).est

3.417414912421342

In [51]:
svd.predict(iid = 10, uid =2).est

3.3969487811532457

In [53]:
#printing the predictions using book id, and the metadata, and giving detailed results
import difflib
import random
movies_file.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [55]:
list(movies_file["original_title"].values)

['Toy Story',
 'Jumanji',
 'Grumpier Old Men',
 'Waiting to Exhale',
 'Father of the Bride Part II',
 'Heat',
 'Sabrina',
 'Tom and Huck',
 'Sudden Death',
 'GoldenEye',
 'The American President',
 'Dracula: Dead and Loving It',
 'Balto',
 'Nixon',
 'Cutthroat Island',
 'Casino',
 'Sense and Sensibility',
 'Four Rooms',
 'Ace Ventura: When Nature Calls',
 'Money Train',
 'Get Shorty',
 'Copycat',
 'Assassins',
 'Powder',
 'Leaving Las Vegas',
 'Othello',
 'Now and Then',
 'Persuasion',
 'La Cité des Enfants Perdus',
 '摇啊摇，摇到外婆桥',
 'Dangerous Minds',
 'Twelve Monkeys',
 'Guillaumet, les ailes du courage',
 'Babe',
 'Carrington',
 'Dead Man Walking',
 'Across the Sea of Time',
 'It Takes Two',
 'Clueless',
 'Cry, the Beloved Country',
 'Richard III',
 'Dead Presidents',
 'Restoration',
 'Mortal Kombat',
 'To Die For',
 'How To Make An American Quilt',
 'Se7en',
 'Pocahontas',
 'When Night Is Falling',
 'The Usual Suspects',
 'Guardian Angel',
 'Mighty Aphrodite',
 'Lamerica',
 'The Big G

In [99]:
def get_movie_id(movie_title, file):
    existing_title = list(file["original_title"].values)
    closest_title = difflib.get_close_matches(movie_title, existing_title) #getting the closest matching movie title from the movies_file
    #here we are using the movie id columns from both files, and based on the id we are getting the movie title from the movies file, then looking fo its closest match
    #then we get the book id based on closes match
    movie_id = file[file["original_title"] == closest_title[0]]#here we are getting the title value which is equal to closestvalue
    return movie_id

In [100]:
get_movie_id("Four Rooms", movies_file)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
17,False,NaN,4000000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",NaN,5,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,...,1995-12-09,4300000.0,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6.5,539.0


In [95]:
def get_movie_id(movie_title, file):
    existing_title = list(file["original_title"].values)
    closest_title = difflib.get_close_matches(movie_title, existing_title) #getting the closest matching movie title from the movies_file
    #here we are using the movie id columns from both files, and based on the id we are getting the movie title from the movies file, then looking fo its closest match
    #then we get the book id based on closes match
    movie_id = file[file["original_title"] == closest_title[0]]["id"].values[0] #getting only id value of the closest match
    return movie_id

#next we write a function to get the movie info from the movie id
def get_movie_info(movie_id, file):
    #getting the info of the movie where th eid from file matches the movie_id, with the matching id
    #getting details like title, and tagline
    movie_info = file[file["id"] == movie_id][["title", "tagline"]]
    #this will give movie info in the form of a dataframe, converting that to a dictionary
    return movie_info

In [96]:
get_movie_info(get_movie_id("Four Rooms", movies_file), movies_file)

,title,tagline
17,Four Rooms,Twelve outrageous guests. Four scandalous requ...


In [103]:
def get_movie_id(movie_title, file):
    existing_title = list(file["original_title"].values)
    closest_title = difflib.get_close_matches(movie_title, existing_title) #getting the closest matching movie title from the movies_file
    #here we are using the movie id columns from both files, and based on the id we are getting the movie title from the movies file, then looking fo its closest match
    #then we get the book id based on closes match
    movie_id = file[file["original_title"] == closest_title[0]]["id"].values[0] #getting only id value of the closest match
    return movie_id

#next we write a function to get the movie info from the movie id
def get_movie_info(movie_id, file):
    #getting the info of the movie where th eid from file matches the movie_id, with the matching id
    #getting details like title, and tagline
    movie_info = file[file["id"] == movie_id][["title", "tagline"]]
    #this will give movie info in the form of a dataframe, converting that to a dictionary
    return movie_info.to_dict(orient = "records")
#finally getting the predictions

def get_predictions(movie_title, file, user_id, model):
    #getting movie id from abve function
    movie_id = get_movie_id(movie_title, file)
    rating = model.predict(iid = movie_id, uid = user_id)
    return rating


In [104]:
get_predictions("Four Rooms", movies_file,400 , model)

Prediction(uid=400, iid='5', r_ui=None, est=3.543608255669773, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [132]:
def get_movie_id(movie_title, file):
    existing_title = list(file["original_title"].values)
    closest_title = difflib.get_close_matches(movie_title, existing_title) #getting the closest matching movie title from the movies_file
    #here we are using the movie id columns from both files, and based on the id we are getting the movie title from the movies file, then looking fo its closest match
    #then we get the book id based on closes match
    movie_id = file[file["original_title"] == closest_title[0]]["id"].values[0] #getting only id value of the closest match
    return movie_id

#next we write a function to get the movie info from the movie id
def get_movie_info(movie_id, file):
    #getting the info of the movie where th eid from file matches the movie_id, with the matching id
    #getting details like title, and tagline
    movie_info = file[file["id"] == movie_id][["title", "tagline"]]
    #this will give movie info in the form of a dataframe, converting that to a dictionary
    return movie_info.to_dict(orient = "records")
#finally getting the predictions

def get_predictions(movie_title, file, user_id, model):
    #getting movie id from abve function
    movie_id = get_movie_id(movie_title, file)
    rating = model.predict(iid = movie_id, uid = user_id).est
    return rating
#setting the threshold values
def get_recommendation(file, user_id, model, thresh = 3.8):
    recommendations = [] # i.e if rating is more than 3.8, then recommend that movie to the user
    movie_titles = list(file["original_title"].values)
    random.shuffle(movie_titles)
    # we need to predict for each movie, hence we need to put the below function under for loop
    for movie_title in movie_titles:
        rating = get_predictions(movie_title, file, user_id, model)
        if rating>=thresh:#if rating is greater than the threshold value than get that movie id, and then return thatmovie info
            movie_id = get_movie_id(movie_title, file)
            recommendations.append(get_movie_info(movie_id, file))
            return recommendations
        

In [133]:
get_recommendation(movies_file, 200, svd)

KeyboardInterrupt: 